In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
import numpy as np

In [3]:
mnist = input_data.read_data_sets('./MNIST_data')
batch_size = 100
width, height = 28, 28
mnist_dim = 28 * 28
random_dim = 10

Extracting ./MNIST_data/train-images-idx3-ubyte.gz


Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
help(mnist.train.next_batch)

Help on method next_batch in module tensorflow.contrib.learn.python.learn.datasets.mnist:

next_batch(batch_size, fake_data=False, shuffle=True) method of tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet instance
    Return the next `batch_size` examples from this data set.



In [5]:
D_W1 = tf.Variable(tf.random_uniform([mnist_dim, 128], -0.05, 0.05))
D_b1 = tf.Variable(tf.zeros(128))
D_W2 = tf.Variable(tf.random_uniform([128, 32], -0.05, 0.05))
D_b2 = tf.Variable(tf.zeros([32]))
D_W3 = tf.Variable(tf.random_uniform([32, 1], -0.05, 0.05))
D_b3 = tf.Variable(tf.zeros([1]))
D_variables = [D_W1, D_b1, D_W2, D_b2, D_W3, D_b3]

In [6]:
G_W1 = tf.Variable(tf.random_uniform([random_dim, 32], -0.05, 0.05))
G_b1 = tf.Variable(tf.zeros([32]))
G_W2 = tf.Variable(tf.random_uniform([32, 128], -0.05, 0.05))
G_b2 = tf.Variable(tf.zeros([128]))
G_W3 = tf.Variable(tf.random_uniform([128, mnist_dim], -0.05, 0.05))
G_b3 = tf.Variable(tf.zeros([mnist_dim]))
G_variables = [G_W1, G_b1, G_W2, G_b2, G_W3, G_b3]

In [7]:
def D(X):
    X1 = tf.nn.relu(tf.matmul(X, D_W1) + D_b1)
    X2 = tf.nn.relu(tf.matmul(X1, D_W2) + D_b2)
    X3 = tf.nn.relu(tf.matmul(X2, D_W3) + D_b3)
    return X3


def G(X):
    X1 = tf.nn.relu(tf.matmul(X, G_W1) + G_b1)
    X2 = tf.nn.relu(tf.matmul(X1, G_W2) + G_b2)
    X3 = tf.nn.relu(tf.matmul(X2, G_W3) + G_b3)
    return X3

In [8]:
x = tf.placeholder(tf.float32, shape=[batch_size, mnist_dim])
z = tf.placeholder(tf.float32, shape=[batch_size, random_dim])

In [26]:
D_loss = -tf.reduce_mean(tf.log(D(x)) + tf.log(1 - D(G(z))))
G_loss = -tf.reduce_mean(tf.log(D(G(z))))

D_opt = tf.train.AdamOptimizer().minimize(D_loss, var_list=D_variables)
G_opt = tf.train.AdamOptimizer().minimize(G_loss, var_list=G_variables)

In [27]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [13]:
from scipy import misc, ndimage

In [28]:
if not os.path.exists('out/'):
    os.makedirs('out/')
epochs = 10000

for e in range(epochs):
    random_noise = list(np.random.normal(size=(batch_size, random_dim)))
    # print(random_noise)
    next_batch, _ = mnist.train.next_batch(batch_size)
    # print(next_batch)
    _, G_loss_ = sess.run([G_opt,
                           G_loss], feed_dict={z: random_noise})
    _, D_loss_ = sess.run([D_opt, D_loss], feed_dict={x: next_batch, z: random_noise})
    if e % 1000 == 0:
        print('epoch %s, D_loss: %s, G_loss: %s' % (e, D_loss_, G_loss_))
        n_rows = 6
        """
        check_imgs = sess.run(random_Y, feed_dict={random_X: random_batch_X}).reshape((batch_size, width, height))[:n_rows*n_rows]
        imgs = np.ones((width*n_rows+5*n_rows+5, height*n_rows+5*n_rows+5))
        for i in range(n_rows*n_rows):
            imgs[5+5*(i%n_rows)+width*(i%n_rows):5+5*(i%n_rows)+width+width*(i%n_rows), 5+5*(i/n_rows)+height*(i/n_rows):5+5*(i/n_rows)+height+height*(i/n_rows)] = check_imgs[i]
        misc.imsave('out/%s.png'%(e/1000), imgs)
        """

epoch 0, D_loss: nan, G_loss: inf


KeyboardInterrupt: 